In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import json

In [30]:
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')
pd.set_option('display.max_columns', None)

In [31]:
with open('../config/lyft_conection.json', 'r') as file:
    credenciales_lyft = json.load(file)

engine = create_engine(f"mysql+pymysql://{credenciales_lyft['USERNAME']}:{credenciales_lyft['PASSWORD']}@{credenciales_lyft['SERVER']}/{credenciales_lyft['DATABASE']}")

### Consultas a base de datos

In [40]:
start_date = '2025-10-01 00:00:00'
end_date = '2025-10-31 23:59:59'

In [41]:
sql_BikeSubscriptionFact_main = f'''

SELECT
    bs.purchaseUser_id,
    bs.start,
    bs.end,
    bs.discountCodeRedemptionNumber,
    bs.member_accountNumber,
    bs.member_birthday,
    bs.member_country,
    bs.member_gender,
    bs.member_postalCode,
    bs.purchasePrice,
    bs.totalPaid,
    bs.status_id,
    bs.subscriptionType_id,
    bs.purchase,
    bm.bikeMemberAttributes_accountNumber,
    bm.bikeMemberAttributes_birthday,
    bm.bikeMemberAttributes_country,
    bm.bikeMemberAttributes_gender,
    bm.bikeMemberAttributes_postalCode,
    bm.email,
    bm.firstName,
    bm.lastName,
    bm.phoneNumber,
    bm.city,
    bs_status.localizedValue0 as status_name,
    bs_type.name_localizedValue1 as subscription_type_name
FROM BikeSubscriptionFact bs
INNER JOIN BikeMemberFact bm 
    ON bs.member_accountNumber = bm.bikeMemberAttributes_accountNumber
INNER JOIN BikeSubscriptionStatusDim bs_status 
    ON bs.status_id = bs_status.id
INNER JOIN BikeSubscriptionTypeDim bs_type 
    ON bs.subscriptionType_id = bs_type.id
WHERE 
    bs.purchase BETWEEN UNIX_TIMESTAMP(CONVERT_TZ("{start_date}", "America/Mexico_City", 'UTC'))*1000
    AND UNIX_TIMESTAMP(CONVERT_TZ("{end_date}", "America/Mexico_City", 'UTC'))*1000 ;

'''

df_sql_BikeSubscriptionFact_main = pd.read_sql(sql_BikeSubscriptionFact_main, engine)
df_sql_BikeSubscriptionFact_main.head()

,purchaseUser_id,start,end,discountCodeRedemptionNumber,member_accountNumber,member_birthday,member_country,member_gender,member_postalCode,purchasePrice,totalPaid,status_id,subscriptionType_id,purchase,bikeMemberAttributes_accountNumber,bikeMemberAttributes_birthday,bikeMemberAttributes_country,bikeMemberAttributes_gender,bikeMemberAttributes_postalCode,email,firstName,lastName,phoneNumber,city,status_name,subscription_type_name
0,NaN,1759410123494,1790946123494,None,147504,4.081752e+11,MX,M,03020,487.93,566.0,0,4,1759410123494,147504,4.081752e+11,MX,M,03020,abraham.ascarraga@gmail.com,Job Abraham,Ascarraga Flores,5524389738,Narvarte Poniente,Active,Anual (I.V.A incluido)
1,NaN,1759423532520,1790959532520,None,217402,-4.257360e+10,MX,F,06500,487.93,566.0,0,4,1759423532520,217402,-4.257360e+10,MX,F,06500,luzzaragoza12@gmail.com,Maria de la Luz,Zaragoza Cruz,5533519046,cuauhtemoc,Active,Anual (I.V.A incluido)
2,NaN,1759449004599,1790985004599,None,258323,5.128056e+11,MX,M,03100,487.93,566.0,0,4,1759449004599,258323,5.128056e+11,MX,M,03100,dielo1@hotmail.com,Diego,Lopez Torreblanca,5524263261,Del Valle Centro,Active,Anual (I.V.A incluido)
3,NaN,1759333533455,1790869533455,None,276589,6.525144e+11,MX,M,03710,487.93,566.0,0,4,1759333533455,276589,6.525144e+11,MX,M,03710,jare0590@gmail.com,Jose Antonio,Rodriguez Elias,55784563,Ciudad de los Deportes,Active,Anual (I.V.A incluido)
4,NaN,1759592652074,1791128652074,None,308321,6.297048e+11,MX,M,15990,487.93,566.0,0,4,1759420804461,308321,6.297048e+11,MX,M,15990,jaimeasl15@hotmail.com,Jaime Alfredo,Sanchez Leon,55520101,None,Active,Anual (I.V.A incluido)


In [43]:
# filtrar por activas
df_sql_BikeSubscriptionFact_main_active = df_sql_BikeSubscriptionFact_main[df_sql_BikeSubscriptionFact_main['status_name'] == 'Active']

# agrupar porsubscription_type_name y contar
gruped = df_sql_BikeSubscriptionFact_main_active.groupby('subscription_type_name').size()
gruped

subscription_type_name
Anual (I.V.A incluido)             13987
Anual Ecobici+ (I.V.A incluido)      334
Ecobici HSBC                         288
dtype: int64

In [33]:
sql_BikeSubscriptionFact = f'''

SELECT
    purchaseUser_id
    start,
    end,
    discountCodeRedemptionNumber,
    member_accountNumber,
    member_birthday,
    member_country,
    member_gender,
    member_postalCode
    purchasePrice,
    totalPaid,
    status_id,
    subscriptionType_id, 
    purchase
FROM BikeSubscriptionFact
WHERE 
	purchase BETWEEN UNIX_TIMESTAMP(CONVERT_TZ("{start_date}", "America/Mexico_City", 'UTC'))*1000
    AND UNIX_TIMESTAMP(CONVERT_TZ("{end_date}", "America/Mexico_City", 'UTC'))*1000 AND subscriptionType_id = 11;

'''

df_sql_BikeSubscriptionFact = pd.read_sql(sql_BikeSubscriptionFact, engine)
df_sql_BikeSubscriptionFact.head(2)

# purchaseUser_id = UserDim
# status_id = BikeSubscriptionStatusDim
# subscriptionType_id = BikeSubscriptionTypeDim

,start,end,discountCodeRedemptionNumber,member_accountNumber,member_birthday,member_country,member_gender,purchasePrice,totalPaid,status_id,subscriptionType_id,purchase


In [ ]:
sql_BikeMemberFact = f'''

SELECT
    id,
    bikeMemberAttributes_accountNumber,
    bikeMemberAttributes_birthday,
    bikeMemberAttributes_country,
    bikeMemberAttributes_gender,
    bikeMemberAttributes_postalCode,
    email,
    firstName,
    lastName,
    phoneNumber,
    city
FROM BikeMemberFact

'''

df_sql_BikeMemberFact = pd.read_sql(sql_BikeMemberFact, engine)
df_sql_BikeMemberFact

,bikeMemberAttributes_accountNumber,bikeMemberAttributes_birthday,bikeMemberAttributes_country,bikeMemberAttributes_gender,bikeMemberAttributes_postalCode,email,firstName,lastName,phoneNumber,city
0,TUWMKRYB,6.260760e+11,US,M,11217,trix@lyft.com,Thomas,Rix,4155686353,Brooklyn
1,7BE3HQMF,6.442200e+11,US,M,90201,lyftpolaris+apple@gmail.com,Lyft,Apple,(650) 499-2804,Los Angeles
2,3PUDW6C8,NaN,,F,None,None,None,None,None,None
3,RQ5QEAE7,5.652504e+11,CA,M,J3B0H4,ndurette@lyft.com,Nikolas,Durette,5142134064,St-Jean-Sur-Richelieu
4,4YNDD843,1.749816e+11,CA,M,H2G2M2,cmontpetit@lyft.com,Claude,Montpetit,(514) 449-5630,Montreal
5,K7BDMCP9,NaN,CA,M,H1E 1X1,caltamirano@lyft.com,Chris,Alt,None,Montreal
6,T2CCFKTK,4.243320e+11,US,O,94904,rmarsan+test1@lyft.com,RJ,Marsan,(630) 936-1800,Kentfield
7,ZH8RQRR4,-2.177928e+11,MX,M,06600,fhebbert@lyft.com,Frank,Hebbert,6174178386,None
8,8WKEMHAU,7.059960e+11,CA,O,H2T 1J5,lbs-aos@lyft.com,Support,Ops,(514) 571-3787,Montreal
9,DCWNUT95,2.146392e+11,MX,M,11950,rarroyo@grupoexpansion.com,Rodrigo,Arroyo,5551014747,Lomas Altas


In [35]:
sql_BikeSubscriptionStatusDim = f'''

SELECT
    id,
    localizedValue0
FROM BikeSubscriptionStatusDim;
'''

df_sql_BikeSubscriptionStatusDim = pd.read_sql(sql_BikeSubscriptionStatusDim, engine)
df_sql_BikeSubscriptionStatusDim.head(2)

,id,localizedValue0
0,0,Active
1,2,To activate


In [36]:
sql_BikeSubscriptionTypeDim = f'''

SELECT
    id,
    name_localizedValue1
FROM BikeSubscriptionTypeDim;
'''

df_sql_BikeSubscriptionTypeDim = pd.read_sql(sql_BikeSubscriptionTypeDim, engine)
df_sql_BikeSubscriptionTypeDim

,id,name_localizedValue1
0,1,1 Día (I.V.A incluido)
1,2,3 Días (I.V.A incluido)
2,3,7 Días (I.V.A incluido)
3,4,Anual (I.V.A incluido)
4,9,Anual Ecobici+ (I.V.A incluido)
5,10,Ecobici HSBC
6,11,ECOBICI vivo
7,9223372036854775807,Inconnu
8,5,Membresía migrada
9,7,Membresía Plus TEST 01 (IVA incluido)
